<a href="https://colab.research.google.com/github/Shivamvats2102/Fake_news_classifier/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
import tensorflow as tf

In [3]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [4]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
data=data.dropna()

In [6]:
data.describe()

,id,label
count,18285.000000,18285.000000
mean,10409.807383,0.433361
std,5998.216236,0.495553
min,0.000000,0.000000
25%,5217.000000,0.000000
50%,10422.000000,0.000000
75%,15603.000000,1.000000
max,20799.000000,1.000000


In [7]:
X=data.drop('label',axis=1)
Y=data['label']

In [8]:
tf.__version__

'2.7.0'

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [10]:
voc_size=10000

One hot Encoding

In [12]:
msg=X.copy()

In [13]:
msg.reset_index(inplace=True)

In [14]:
import nltk
import re
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(msg)):
    review = re.sub('[^a-zA-Z]', ' ', msg['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[5429, 5553, 5906, 2912, 8768, 1623, 4863, 1933, 9719, 431],
 [8046, 5832, 9047, 9256, 8214, 2902, 6815],
 [6665, 6463, 9246, 1117],
 [8872, 2606, 7486, 1364, 2094, 7086],
 [2084, 8214, 2579, 6992, 5254, 3117, 8214, 7645, 4841, 5872],
 [5137,
  499,
  8766,
  5881,
  616,
  6815,
  597,
  6989,
  6128,
  6976,
  2489,
  2289,
  4218,
  7025,
  6815],
 [5928, 6716, 2601, 3381, 9894, 9719, 3786, 950, 5666, 2619, 1213],
 [2480, 4080, 7888, 3538, 9901, 4879, 6815, 7242, 5666, 2619, 1213],
 [1916, 3845, 1164, 2229, 3398, 7523, 1854, 6732, 6815, 3355],
 [245, 4772, 8623, 424, 4692, 4433, 638, 9401],
 [9131, 6610, 6585, 7747, 3683, 3273, 5868, 662, 3327, 4750, 4425],
 [1364, 533, 8768, 7523, 6815, 9901],
 [8296, 9074, 5955, 1048, 2473, 6740, 2560, 4029, 2057],
 [8524, 8368, 3595, 2578, 996, 797, 2740, 5666, 2619, 1213],
 [8717, 419, 1363, 328, 3557, 5666, 2619, 1213],
 [5531, 2545, 3145, 614, 3543, 7339, 6621, 3111, 3872, 7235],
 [1712, 2491, 5832],
 [4316, 8913, 908, 8405, 6815, 5484, 1140,

In [18]:
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
# print(embedded_docs)

In [19]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 5429, 5553, 5906, 2912, 8768, 1623, 4863,
       1933, 9719,  431], dtype=int32)

In [47]:
from keras.layers import Dropout

In [49]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 25, 40)            400000    
                                                                 
 dropout_1 (Dropout)         (None, 25, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
X_final=np.array(embedded_docs)
y_final=np.array(Y)
y_final

array([1, 0, 1, ..., 0, 1, 1])

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [52]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 44ms/step - loss: 0.3559 - accuracy: 0.8312 - val_loss: 0.2145 - val_accuracy: 0.9079
Epoch 2/10
192/192 [==============================] - 8s 41ms/step - loss: 0.1609 - accuracy: 0.9356 - val_loss: 0.1968 - val_accuracy: 0.9178
Epoch 3/10
192/192 [==============================] - 8s 41ms/step - loss: 0.1079 - accuracy: 0.9600 - val_loss: 0.2175 - val_accuracy: 0.9181
Epoch 4/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0744 - accuracy: 0.9729 - val_loss: 0.2446 - val_accuracy: 0.9130
Epoch 5/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0500 - accuracy: 0.9827 - val_loss: 0.2443 - val_accuracy: 0.9100
Epoch 6/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0398 - accuracy: 0.9862 - val_loss: 0.3255 - val_accuracy: 0.9104
Epoch 7/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0278 - accuracy: 0.9915 - val_loss: 0.4948 - val_accuracy: 0.891

In [53]:
len(embedded_docs)

18285

In [54]:
y_pred=model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [55]:
from sklearn.metrics import confusion_matrix

In [56]:
confusion_matrix(y_test,y_pred)

array([[3118,  301],
       [ 281, 2335]])

In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9035625517812759